# Kubernetes Cluster Versions Report

This report loops through all the clusters we maintain, and reports their master and node versions

In [1]:
from deployer.helm_upgrade_decision import get_all_cluster_yaml_files
from deployer.cluster import Cluster
import subprocess
import json
from tabulate import tabulate

from ruamel.yaml import YAML
yaml = YAML(typ='safe')

In [2]:
cluster_files = get_all_cluster_yaml_files()

clusters = []
for cf in cluster_files:
    with open(cf) as f:
        cluster = Cluster(yaml.load(f), cf.parent)
    clusters.append(cluster)

In [3]:
versions = {}
for c in clusters:
    with c.auth():
        cluster_name = c.spec['name']
        # Populate cloud provider type
        versions[cluster_name] = {'provider': c.spec['provider']}
        
        # Get master version
        version_output = json.loads(subprocess.check_output(['kubectl', 'version', '--output', 'json']))
        server_version = version_output["serverVersion"]["gitVersion"]
        versions[cluster_name]['master'] = server_version.split('-')[0]
        
        # Get node versions
        node_output = json.loads(subprocess.check_output(["kubectl", "get", "node", "--output", "json"]))
        node_versions = set([n["status"]["nodeInfo"]["kubeletVersion"].split('-')[0] for n in node_output["items"]])
        versions[cluster_name]['nodes'] = node_versions
        
        

In [4]:
headers = ('provider', 'cluster', 'master version', 'node version(s)')
tabulate([
    (version['provider'], name, version['master'], ','.join(version['nodes']))
    for name, version in versions.items()
], headers, tablefmt="html")


provider,cluster,master version,node version(s)
aws,smithsonian,v1.25.9,v1.25.7
gcp,awi-ciroh,v1.25.8,v1.25.6
aws,jupyter-meets-the-earth,v1.25.9,v1.25.7
aws,victor,v1.25.9,v1.25.6
gcp,linked-earth,v1.24.12,v1.24.7
gcp,2i2c,v1.26.4,v1.26.4
aws,carbonplan,v1.24.13,v1.24.7
aws,nasa-veda,v1.25.9,v1.25.6
kubeconfig,utoronto,v1.26.3,v1.26.3
aws,ubc-eoas,v1.24.13,v1.24.7
